[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openvinotoolkit/openvino_tensorflow/blob/master/docs/cloud_instructions/Colab_OpenVINO_TensorFlow_classification_example.ipynb)

## INSTALL OpenVINO™ integration with TensorFlow


In [ ]:
# Upload the required wheel files, models and images in a google drive folder
# Uncomment and run the below command to copy them in your current workspace
#!cp /content/drive/MyDrive/TF-OV/working_dir_files/* . 

!pip -q install --upgrade pip
!pip -q install pillow

# Install stock TensorFlow
!pip -q install tensorflow==2.7.0 

# Install OpenVINO™ integration with TensorFlow
!pip -q install openvino-tensorflow

In [ ]:
!ldd --version
!python3 --version

In [ ]:
!git clone --quiet https://github.com/openvinotoolkit/openvino_tensorflow.git
%cd openvino_tensorflow
!git submodule init
!git submodule update --recursive


# Now lets infer

In [90]:
from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.display import HTML

import os
import numpy as np
import tensorflow as tf
import openvino_tensorflow as ovtf
import tensorflow_hub as hub

from PIL import Image
import time

In [101]:
def preprocess_image(file_name,
               input_height=299,
               input_width=299,
               input_mean=0,
               input_std=255):
    """Reads input image file, resize it to given input height and width.
       Preprocesses input image using Keras api and returns the image
    """  
    image = Image.open(file_name)
    resized_image = image.resize((input_height,input_width))
    resized_image = np.asarray(resized_image, np.float32)
    normalized_image = (resized_image - input_mean) / input_std
    result = np.expand_dims(normalized_image, 0)
    return result

In [102]:
def load_labels(label_file):
    """Parses the label file, assuming that labels are separated with a newline
       in the file and returns the list of labels.
    """  
    label = []
    proto_as_ascii_lines = tf.io.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

In [103]:
def infer_openvino_tensorflow(model_file, file_name , input_height, input_width, input_mean, input_std, label_file):
    """Takes the tensorflow model and all other input parameters as arguments. 
       Run inference on the classification model and prints the predictions.
    """
    print("CREATE MODEL - BEGIN")
    if model_file=="":
      model = hub.load("https://tfhub.dev/google/imagenet/inception_v3/classification/4")
    else:
      model = tf.saved_model.load(model_file)
    print("CREATE MODEL - END")

    print("PREDICTION - BEGIN") 

    img =  tf.convert_to_tensor(preprocess_image(
            file_name, input_height=input_height, input_width=input_width, input_mean=input_mean, input_std=input_std))

    # Warmup
    results = model(img)
    # Run
    
    for num_times in range(10):
        start = time.time()
        results = model(img)
        elapsed = time.time() - start
        print('Inference time in ms: %f' % (elapsed * 1000))
            
    print("PREDICTION - END")
    results = tf.nn.softmax(results).numpy()
    
    if label_file:
        top_5 = tf.argsort(results, axis=-1, direction="DESCENDING")[0][:5].numpy()
        labels = load_labels(label_file)
        for i,item in enumerate(top_5):
            print(labels[item], results[0][top_5][i])
    else:
        print("No label file provided. Cannot print classification results")

In [ ]:
file_name = tf.keras.utils.get_file(
    'grace_hopper.jpg',
    "https://www.tensorflow.org/images/grace_hopper.jpg")
model_file = ""
label_file = tf.keras.utils.get_file(
    'ImageNetLabels.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
input_height = 299
input_width = 299
input_mean = 0
input_std = 255
backend_name = "CPU"
    

#Print list of available backends
print('Available Backends:')
backends_list = ovtf.list_backends()
for backend in backends_list:
    print(backend)
ovtf.set_backend(backend_name)

print("OpenVINO TensorFlow is enabled")
infer_openvino_tensorflow(model_file, file_name, input_height, input_width, input_mean, input_std, label_file)

In [ ]:
#Disable
ovtf.disable()
print("")
print("")
print("OpenVINO TensorFlow is disabled")
infer_openvino_tensorflow(model_file, file_name, input_height, input_width, input_mean, input_std, label_file )
ovtf.enable()
